In [ ]:
def identify_collinear(corr_matrix,correlation_threshold,X):
    
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
    
    # Select the features with correlations above the threshold
    # Need to use the absolute value
    to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]
    
    # Dataframe to hold correlated pairs
    record_collinear = pd.DataFrame(columns = ['corr_feature1', 'corr_feature2', 'corr_value'])
    
    # Iterate through the columns to drop to record pairs of correlated features
    for column in to_drop:
    
        # Find the correlated features
        corr_features = list(upper.index[upper[column].abs() > correlation_threshold])
    
        # Find the correlated values
        corr_values = list(upper[column][upper[column].abs() > correlation_threshold])
        drop_features = [column for _ in range(len(corr_features))]    
    
        # Record the information (need a temp df for now)
        temp_df = pd.DataFrame.from_dict({'corr_feature1': drop_features,
                                         'corr_feature2': corr_features,
                                         'corr_value': corr_values})
    
        # Add to dataframe
        record_collinear = record_collinear.append(temp_df, ignore_index = True)
    drops=[]## 下面是删除规则
    if record_collinear.shape[0]==0:
        return 'nothing!','nothing!'
    for i,j in zip(record_collinear.corr_feature1.values.tolist(),record_collinear.corr_feature2.values.tolist()):
        if X[i].nunique()>X[j].nunique():
            drops.append(j)
        else:
            drops.append(i)
    drops=list(set(drops))
    return record_collinear,drops

In [ ]:
def lgb_model(params=None,cv=KFold(5),X=None,y=None,early_stopping_rounds=200,cols=None,categorical_feature=None,group_feature=None):

    #folds = StratifiedKFold(n_splits=NFOLDS)
    folds =cv
    if cols==None:
        columns = list(X.columns)
    else:
        columns=cols
    if group_feature:
        columns.remove(group_feature)
        splits = folds.split(X[columns], y,groups=X[group_feature])
    else:
        splits = folds.split(X[columns], y)
    y_oof = np.zeros(X.shape[0])
    score = 0


    clfs=[]
    for fold_n, (train_index, valid_index) in enumerate(splits):
        X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]


        clf = lgb.LGBMClassifier(**params)
        if categorical_feature:
            clf.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_valid,y_valid)], \
                     eval_metric='auc', verbose=100, early_stopping_rounds=early_stopping_rounds,categorical_feature=categorical_feature)
        else:
            clf.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_valid,y_valid)], \
                     eval_metric='auc', verbose=100, early_stopping_rounds=early_stopping_rounds)

        y_pred_valid = clf.predict_proba(X_valid)[:,1]
        y_oof[valid_index] = y_pred_valid
        print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")

        score += roc_auc_score(y_valid, y_pred_valid) / 6

        del X_train, X_valid, y_train, y_valid
        gc.collect()
        clfs.append(clf)

    print(f"\nMean AUC = {score}")
    print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")
    return clfs

In [ ]:
#使用相关系数对交叉后的特征进行粗筛
total_cat=imp_cat+cross_cat
corr_spearman=X[total_cat].rank().corr()
corr_pearson=X[total_cat].corr()

models=[]
for thereshold in [0.79999,0.899999,0.989999]:
    _,drops_spearman=identify_collinear(corr_spearman,thereshold,X)#需要用到共线性识别模块
    _,drops_pearson=identify_collinear(corr_pearson,thereshold,X)
    drops=list(set(drops_spearman+drops_pearson))
    cols=list(X.columns)
    cols.remove('TransactionDT')
    for col in drops:
        cols.remove(col)
    #用到五折lgb模块
    clfs=lgb_model(params=params,cv=KFold(5),X=X,y=y,early_stopping_rounds=200,cols=cols,categorical_feature=None,group_feature=None)
    y_pred=np.zeros(X_test.shape[0])
    for clf in clfs:
        y_pred+=clf.predict_proba(X_test[cols])[:,1]/5
    models.append(clfs)
    del cols,clfs,y_pred,sub
    gc.collect()